In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import pypsa
import calendar
import re
import pickle
from pathlib import Path

from _helpers import configure_logging, get_scenario
import logging
logger = logging.getLogger(__name__)

from _helpers import calculate_annual_investment

In [ ]:
if __name__ == "__main__":

    scenario = get_scenario(snakemake)
    configure_logging(snakemake)

    supply = xr.open_dataset(snakemake.input["supply"])
    
    # Load additional components
    with open(snakemake.input["additional_components"], "rb") as f:
        override_component_attrs = pickle.load(f)
    
    network = pypsa.Network(override_component_attrs=override_component_attrs)
    network.import_from_netcdf(snakemake.input['network'])
    
    Path(snakemake.output['lcoes']).parent.mkdir(parents=True, exist_ok=True)
    
    # Handle leap years by removing 29th of February
    supply = supply.sel(time=~((supply.time.dt.month == 2) & (supply.time.dt.day == 29)))

    # Load annual demand (consider overwrite option in config)
    if scenario["synthetic_demand"].lower() in ['gegis','custom']:
        demand = pd.read_csv(snakemake.input["demand"], sep=';', index_col='region')

        demand = demand.loc[snakemake.wildcards['from'], 'demand [GWh]']
        demand *= 1e3 # in MWh
        
    else:
        assert False, "Option in config for scenario -> synthethic_demand unknown."
    

    # Apply scenario modifier
    demand *= scenario['modifiers']['synthetic_demand']
    logger.info(f'Removing {demand} MWh of annual demand from potential supply.')
        
    # Remove unused variables
    supply = supply.sel(region=snakemake.wildcards["from"], drop=True).fillna(0.)
    exclude_vars = list(filter(
                            lambda x: re.search("csp|density|pvrooftop|overlap", x), 
                            supply.data_vars))

    supply = supply.drop_vars(exclude_vars).drop_dims(["class_1","class_2","dim_0"])

    capacities = supply[list(filter(lambda x: "capacity" in x, supply.data_vars))]

    capacities = capacities.rename_vars({d:d.replace("capacity_","") for d in capacities})
    capacities = capacities.rename_vars({d:f"wind{d}" for d in ["offshore","onshoreA","onshoreB"]})

    # capacities from GEGIS are in GW, continue using MW
    capacities *= 1e3

    capacity_factors = supply[list(filter(lambda x: "CFtime" in x, supply.data_vars))]

    capacity_factors = capacity_factors.rename_vars({d:d.replace("CFtime_","") for d in capacity_factors})

    # Determine which generation capacities are available for export
    # and which are needed for satisfying domestic electricity demand
    # Simple heuristic:
    # 1.) Consider annual demand and generation per source+source class
    # 2.) Reserve (Exclude) sources with lowest LCoE for demand
    # 3.) Source + source class where demand and generation overlap has
    #     its generation capacitiy partially reduced based on share of overlap.

    # 1.) name mapping between dataset and cost.csv file for technologies
    mapping = {
        "windoffshore":"offwind",
        "windonshoreA":"onwind",
        "windonshoreB":"onwind",
        "pvplantA":"solar-utility",
        "pvplantB":"solar-utility",
              }

    wacc = pd.read_csv(snakemake.input['wacc'], comment='#', index_col="region")
    wacc = wacc.loc[snakemake.wildcards["from"], scenario["wacc"]]
    wacc *= scenario['modifiers']['wacc'] # Apply scenario modifier

    def calculate_investment(x):

        name = x.name
        cost_name = mapping[name]

        invest = calculate_annual_investment(cost_name, wacc, snakemake.input["costs"])
        invest *= 1.e3 # Investment costs in costs.data in kW, here MW and MWh
        
        return x * invest

    investment = capacities.apply(calculate_investment)

    # Calculate annual generation
    generation = (capacity_factors*capacities).fillna(0.).sum(dim="time")

    # Calculate LCoE
    lcoe = (investment/generation)

    # Convert capacities and LCoE into an easier to handle format
    df = xr.merge([
            capacities.to_array().rename("capacity").rename({"variable":"technology"}),
            lcoe.to_array().rename("lcoe").rename({"variable":"technology"}),
            generation.to_array().rename("annual generation").rename({"variable":"technology"})
         ]).to_dataframe()
    df = df.dropna().sort_values("lcoe")

    # Calculate annual generation across all sources + source classes
    df["cumulative generation"] = df.cumsum()["annual generation"]

    # Remove inf values (espc. for LCoE where generation = 0)
    df = df.replace([np.inf,-np.inf],np.nan).dropna()

    # Save LCoEs as intermediary result
    df.to_csv(snakemake.output['lcoes'])

    ## Remaining demand overlapping into higher source class
    
    fully_reserved_capacity = df[df["cumulative generation"] <= demand]
    if fully_reserved_capacity.empty: # regions with lowest class outproducing demand
        fully_reserved_capacity = 0
    else: # regions with some classes fully reserved for domestic demand
        fully_reserved_capacity = fully_reserved_capacity.iloc[-1]["cumulative generation"]
        
    residual_demand = demand - fully_reserved_capacity

    # 2.) Exclude sources reserved for domestic demand
    df = df[df["cumulative generation"] > demand]

    # 3.) Reduce capacitiy at overlap
    residual_demand_share = residual_demand/df.iloc[0]["annual generation"]

    df.iloc[0]["capacity"] *= (1.-residual_demand_share)

    # Add filtered list of generators to network

    carrier_mapping = {"windonshoreA":"wind","windonshoreB":"wind",
                       "windoffshore":"wind",
                       "pvplantA":"pv","pvplantB":"pv"}    
    
    df = df.reorder_levels(['technology','class']).sort_index()
    for idx, row in df.iterrows():
        
        p_max_pu = supply[f"CFtime_{idx[0]}"].sel({"class":idx[1]}, drop=True).to_pandas()
    
        # Lower and upper clipping
        # max 100% (higher values due to EPS)
        # min 0.1% or 0. (easier for solving and non-relevant generation)
        p_max_pu = p_max_pu.clip(upper=1.).where(lambda x: x > 0.0001, 0).to_numpy()
        
        network.add("Generator",
                    name=f"{idx[0]} {idx[1]}",
                    bus="electricity (exp)",
                    p_nom_max=row['capacity'],
                    p_nom_extendable=True,
                    p_max_pu=p_max_pu,
                    # Cost data for kW, here MW implicitly assumed
                    capital_cost=1.e3*calculate_annual_investment(mapping[idx[0]], wacc, snakemake.input["costs"]),
                    carrier=carrier_mapping[idx[0]]
                   )

    network.export_to_netcdf(snakemake.output['network'])
    
    supply.close()